# 学习 PyTorch-Day2

In [5]:
import torch
import numpy as np

device = torch.device("mps")

## Bridge with Numpy

- CPU上的Tensor可以与Numpy数组共享底层内存，改变其中一个即可改变另一个

### Tensor 转成 Numpy 数组

In [3]:
t = torch.ones(5)
n = t.numpy()
print(t)
print(n)

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]


In [4]:
# 改变Tensor，观察Numpy数组是否变化
t.add_(2)
print(t)
print(n)

tensor([3., 3., 3., 3., 3.])
[3. 3. 3. 3. 3.]


### Numpy数组 转成 Tensor

In [6]:
n = np.ones(5)
t = torch.from_numpy(n)
print(n)
print(t)

# 修改Numpy数组，观察Tensor是否变化
np.add(n, 3, out=n)
print(n)
print(t)

[1. 1. 1. 1. 1.]
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
[4. 4. 4. 4. 4.]
tensor([4., 4., 4., 4., 4.], dtype=torch.float64)
